In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np
from sklearn.manifold import TSNE
import plotly.io as pio

# Configurar o renderer
pio.renderers.default = "iframe"

# 1. Garantir que as embeddings dos documentos e da query estejam no formato correto
docs_embeddings_np = docs_embeddings  # Renomear para manter consistência
query_embedding_np = query_embedding.reshape(1, -1)  # Garantir formato correto

# 2. Combinar as embeddings dos documentos e a embedding da query
all_embeddings = np.vstack([docs_embeddings_np, query_embedding_np]).astype('float32')

# 3. Aplicar t-SNE para reduzir para 2 dimensões
tsne = TSNE(n_components=2, random_state=42, perplexity=10, n_iter=500)
embeddings_2d = tsne.fit_transform(all_embeddings)

# 4. Criar lista de textos (documentos + query)
texts = [doc.page_content for doc in docs]  # Documentos
query_text_wrapped = "QUERY: " + query_text  # Texto específico para a query
texts.append(query_text_wrapped)  # Garantir que a query seja a última entrada

# 5. Criar DataFrame para visualização
data = pd.DataFrame({
    "x": embeddings_2d[:, 0],  # Primeiro componente t-SNE
    "y": embeddings_2d[:, 1],  # Segundo componente t-SNE
    "Text": texts,             # Textos dos documentos e da query
    "Type": ["Document"] * len(docs) + ["Query"]  # Tipo: Document ou Query
})

# 6. Formatar os tooltips para limitar a largura
def format_tooltip(text, max_length=50):
    """Quebra o texto em múltiplas linhas com um limite de caracteres."""
    return '<br>'.join([text[i:i+max_length] for i in range(0, len(text), max_length)])

data["FormattedText"] = data["Text"].apply(lambda x: format_tooltip(x))

# 7. Criar gráfico interativo com Plotly
fig = px.scatter(
    data,
    x="x",
    y="y",
    color="Type",
    hover_data={"FormattedText": True, "x": False, "y": False},  # Tooltips formatados
    title="t-SNE dos Documentos e Query Embeddings",
    labels={"x": "t-SNE Component 1", "y": "t-SNE Component 2"}
)

# Melhorar a aparência dos pontos
fig.update_traces(marker=dict(size=10, opacity=0.8, line=dict(width=1, color='DarkSlateGrey')))

# 8. Salvar como HTML
fig.write_html("tsne_plot_tooltip.html")
print("Gráfico salvo como 'tsne_plot_tooltip.html'. Abra no navegador para visualizar.")